In [2]:
# Imported dask and check on cluster

import dask.dataframe as dd
from dask.distributed import Client

client = Client(n_workers=16)
client

Client Scheduler: tcp://127.0.0.1:43695 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 16 Cores: 16 Memory: 67.53 GB


In [3]:
# Loaded all the csv files
df = dd.read_csv('*.csv')
df

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
npartitions=5,,,,,
,object,object,object,object,int64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [4]:
df.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [10]:
# Number of rows
print (len(df.index))

1956


In [11]:
# Number of columns
len(df.columns) 

5

In [12]:
# The value counts for class 1 (spam) is 1005 and class 0 
# (legitmate) is 951
df['CLASS'].value_counts().compute()

1    1005
0     951
Name: CLASS, dtype: int64

In [13]:
# Created a lowercase comments column by transforming the content
# column into all lowercase
df['lowercase_comments'] = df.CONTENT.astype(str).str.lower()

In [14]:
df.lowercase_comments.head()

0    huh, anyway check out this you[tube] channel: ...
1    hey guys check out my new channel and our firs...
2               just for test i have to say murdev.com
3     me shaking my sexy ass on my channel enjoy ^_^ ﻿
4              watch?v=vtarggvgtwq   check this out .﻿
Name: lowercase_comments, dtype: object

In [15]:
# Created a column for check with a boolean test if the lowercase
# comments column had a "check" in it
df['check'] = df['lowercase_comments'].str.contains('check')

In [16]:
df.check.head()

0     True
1     True
2    False
3    False
4     True
Name: check, dtype: bool

In [17]:
# Value counts for users that had "check" in their comments
df['check'].value_counts().compute()

False    1476
True      480
Name: check, dtype: int64

In [19]:
# Could not find a better way than to just groupby and value count
# The count of class 0 (legitimate comments) that includes "check" is 19
# whereas the count of class 1 (spam comments) that includes "check" is 461 

df.groupby(['CLASS']).check.apply(lambda x: x.value_counts()).compute()   

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  if __name__ == '__main__':


CLASS       
0      False    932
       True      19
1      False    544
       True     461
Name: check, dtype: int64

Big Data Options

As many things continue to be digitized, there is an increasing rate of data collection and thus larger size of datasets to be processed.  In response, the technology and software industry and community continue to innovate and develop numerous tools and techniques to efficiently and effectively process the big data. Computation power in terms of processing speed and memory management are becoming key operational advantages in product or service work flows. Without the capacity to process big data, a company can fall behind with competitive analysis and insight that drives their service or product. Before cloud computing technology became more cost effective, the Hadoop and MapReduce method were ground breaking to help scale out data with clusters for users to operate multiple computer resources individually. The progress in parallel processing was a major step forward that improved efficiency by processing rows simultaneously and across clusters while minimizing the use of RAM when possible. These advancements “under the hood” provided data scientists and analysts the ability to use heavy computation models and larger datasets without wasting as much time. AWS and the other giants in cloud computing have now provided a more cost-effective way to scale up. With the ability to essentially rent higher computing power and much more RAM on demand, companies can now perform many big data and machine learning analysis without utilizing clusters or heavy up-front fixed investment in larger servers. There are still certain processes whereby clustering is not as efficient or effective and with cloud computing cost becoming more “affordable” and trust rising with centralized security from the major companies (in some cases), we may see even higher usage of big data resources. 

From my limited experience this week, the notebook platforms were all good options. SageMaker’s computing power is very impressive if I had the resources. Databricks was great for a free resource where the additional resources are needed through clustering but likely more appropriate for a learning tool and side projects than a reliable all-in-one production platform for work. Spark was also fairly seamless to use with many functionalities and a UX that resembled the small data python/numpy/pandas environment. Dask and Numba are great libraries that can be easily used by data scientists that are used to the standard python small data environment noted earlier while performing “smart” processing behind the scene to make things much more efficient respectively by delaying computation or compiling information and processing with more efficient machine code when possible. As a user I can focus on what I need returned in an output while Dask and Numba finds a way to do it best without explicitly instructed. If I worked for a larger company with more resources and big data analysis is key to the job, then AWS and Spark would be the optimal choice. If my budget is more limited and large datasets are not a majority of my work flow, then a combo of Databricks and Dask may be sufficient for ad-hoc tasks. 
